In [ ]:

!pip install --upgrade pip
!pip install pymap3d==2.1.0
!pip install -U l5kit


In [ ]:

import numpy as np
import os
import torch

from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18, resnet50, resnet34
from torchvision.models import vgg19
from tqdm import tqdm
from typing import Dict

from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer

from torch.utils.data.dataset import Subset
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS


In [ ]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import matplotlib.pyplot as plt

import os
import random
import time

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display
from tqdm import tqdm_notebook
import gc, psutil

print(l5kit.__version__)

## Memory information

In [ ]:
def memory(verbose=True):
    mem = psutil.virtual_memory()
    gb = 1024*1024*1024
    if verbose:
        print('Physical memory:',
              '%.2f GB (used),'%((mem.total - mem.available) / gb),
              '%.2f GB (available)'%((mem.available) / gb), '/',
              '%.2f GB'%(mem.total / gb))
    return (mem.total - mem.available) / gb

def gc_memory(verbose=True):
    m = gc.collect()
    if verbose:
        print('GC:', m, end=' | ')
        memory()

memory();

### Input path and train param

In [ ]:
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"
DEBUG = False
VALIDATION = True

## Config information

In [ ]:

cfg = {
    'format_version': 4,
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name' : 'resnet34_gru'
    },
    
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False
    },
    
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },
    
    'valid_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },

    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 16,
        'shuffle': False,
        'num_workers': 4
    },
    
    'train_params': {
        'max_num_steps': 100 if DEBUG else 12000,
        'checkpoint_every_n_steps': 1000,
        'eval_every_n_steps': 1000,
        'steps': 12000,
        'update_steps': 100,
        'checkpoint_steps': 3000,
        # 'eval_every_n_steps': -1
    }
}


### environment variable

In [ ]:
# set env variable for data

os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)

### rasterizer

In [ ]:
%%time
# ===== INIT DATASET

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

## Train data loader

In [ ]:
%%time
# Train dataset
train_cfg = cfg["train_data_loader"]
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open(cached=False)  # to prevent run out of memory
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"],
                              batch_size=train_cfg["batch_size"], num_workers=train_cfg["num_workers"],
                              pin_memory=True
                             )
print(train_dataset)

## Validation data loader

In [ ]:
%%time
# Train dataset
valid_cfg = cfg["valid_data_loader"]
valid_zarr = ChunkedDataset(dm.require(valid_cfg["key"])).open(cached=False)  # to prevent run out of memory
valid_dataset = AgentDataset(cfg, valid_zarr, rasterizer)
valid_dataloader = DataLoader(valid_dataset, shuffle=train_cfg["shuffle"],
                              batch_size=train_cfg["batch_size"], num_workers=valid_cfg["num_workers"],
                              pin_memory=True
                             )
print(train_dataset)

## Negative log likelihood loss function

In [ ]:

# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

## Resnet-GRU model

In [ ]:

class GRU_layer(nn.Module):
    def __init__(self, device, batch=32, in_dim=512, out_dim=1024, hidden_size=2048):
        super().__init__()
        self.batch = batch
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hidden_temp = torch.zeros(2, batch, hidden_size).to(device)
            
        self.GRU_layer_1 = nn.GRU(in_dim, hidden_size, batch_first=True, bidirectional=True).to(device)
        self.GRU_layer_2 = nn.GRU(out_dim, hidden_size, batch_first=True, bidirectional=True).to(device)
        
        self.seq_1 = nn.Sequential(nn.Linear(in_features=2*hidden_size, out_features=2048),
                            nn.ReLU(inplace=True),nn.Linear(2048, out_dim).to(device) )
        
            
    def forward(self, x):

        out_1, hidden_state_1 = self.GRU_layer_1(x.view(self.batch, 1, self.in_dim), self.hidden_temp)
        out_1 = self.seq_1(out_1)
        
        out_2, hidden_state_2 = self.GRU_layer_2(out_1, hidden_state_1)
        out_2 = self.seq_1(out_2)
        
        out_3, hidden_state_3 = self.GRU_layer_2(out_2, hidden_state_2)
        out_3 = self.seq_1(out_3)
        
        return out_3


class Resnet_network(nn.Module):

    def __init__(self, cfg: Dict, device, num_modes=3):
      
        super().__init__()
        architecture = cfg["model_params"]["model_architecture"]
        backbone = eval(architecture)(pretrained=True, progress=True)
        self.backbone = backbone
        
        num_history_channels = (
            cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels
        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )

        backbone_out_features = 512

        self.future_len = cfg["model_params"]["future_num_frames"]
        self.batch_size = cfg['train_data_loader']['batch_size']
        num_targets = 2 * self.future_len
        
        self.num_preds = num_targets * num_modes
        
        self.num_modes = num_modes

        self.gru_decoder = GRU_layer(
            device,
            batch=self.batch_size, 
            in_dim=backbone_out_features
            )
        
        self.logit = nn.Linear(1024, out_features=self.num_preds + num_modes)

    def forward(self, x, criterion=pytorch_neg_multi_log_likelihood_batch):

        x = self.backbone.conv1(x)

        x = self.backbone.bn1(x)

        x = self.backbone.relu(x)

        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)

        x = self.backbone.layer2(x)

        x = self.backbone.layer3(x)

        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)

        x = torch.flatten(x, 1)
       
        x = self.gru_decoder(x)

        x = x.squeeze(1)

        x = self.logit(x)
        
        preds, confidences = torch.split(x, self.num_preds, dim=1)
        preds = preds.view(self.batch_size, self.num_modes, self.future_len, 2)
        confidences = torch.softmax(confidences, dim=1)
        
        return preds, confidences

## Model and optimizer initiation

In [ ]:
# ==== INIT MODEL
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Resnet_network(cfg, device)

#if you need to load pretrained models, uncomment the following lines

'''
weight_path = '/kaggle/input/rmse-model/resnet34_gru_final.pth'
model.load_state_dict(torch.load(weight_path))
'''

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### calling model, using negative loss likelihood loss function

In [ ]:
def forward(data, model, device, criterion=pytorch_neg_multi_log_likelihood_batch, compute_loss=True):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    preds, confidences = model(inputs)
    # skip compute loss if we are doing prediction
    loss = criterion(targets, preds, confidences, target_availabilities) if compute_loss else 0
    return loss, preds, confidences

## calling model, using rMSE on Neg log likelihood 

In [ ]:
'''
def forward(data, model, device, criterion=pytorch_neg_multi_log_likelihood_batch, compute_loss=True):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    preds, confidences = model(inputs)
    # skip compute loss if we are doing prediction
    nll = criterion(targets, preds, confidences, target_availabilities) if compute_loss else 0
    loss = torch.sqrt(2 * nll / 50)
    return loss, preds, confidences
'''

## training loop

In [ ]:
%%time

tr_it = iter(train_dataloader)
n_steps = cfg["train_params"]["steps"]
progress_bar = tqdm_notebook(range(1, 1 + n_steps), mininterval=5.)
losses = []
iterations = []
metrics = []
memorys = []
times = []
model_name = cfg["model_params"]["model_name"]
update_steps = cfg['train_params']['update_steps']
checkpoint_steps = cfg['train_params']['checkpoint_steps']
t_start = time.time()
torch.set_grad_enabled(True)

for i in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    model.train()
    loss, _, _ = forward(data, model, device)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_v = loss.item()
    losses.append(loss_v)

    if i % update_steps == 0:
        mean_losses = np.mean(losses)
        timespent = (time.time() - t_start) / 60
        print('i: %5d'%i,
              'loss: %10.5f'%loss_v, 'loss(avg): %10.5f'%mean_losses, 
              '%.2fmins'%timespent, end=' | ')
        mem = memory()
        if i % checkpoint_steps == 0:
            torch.save(model.state_dict(), f'{model_name}_{i}.pth')
            torch.save(optimizer.state_dict(), f'{model_name}_optimizer_{i}.pth')
        iterations.append(i)
        metrics.append(mean_losses)
        memorys.append(mem)
        times.append(timespent)

torch.save(model.state_dict(), f'{model_name}_final.pth')
torch.save(optimizer.state_dict(), f'{model_name}_optimizer_final.pth')
results = pd.DataFrame({
    'iterations': iterations, 
    'metrics (avg)': metrics,
    'elapsed_time (mins)': times,
    'memory (GB)': memorys,
})
results.to_csv(f'train_metrics_{model_name}_{n_steps}.csv', index=False)
print(f'Total training time is {(time.time() - t_start) / 60} mins')
memory()
display(results)

## Visualization of training metrics

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(results['iterations'], results['metrics (avg)'])
plt.xlabel('steps'); plt.ylabel('metrics (avg)')
plt.grid(); plt.show()

plt.figure(figsize=(12, 4))
plt.plot(results['iterations'], results['memory (GB)'])
plt.xlabel('steps'); plt.ylabel('memory (GB)')
plt.grid(); plt.show()

plt.figure(figsize=(12, 4))
plt.plot(results['iterations'], results['elapsed_time (mins)'])
plt.xlabel('steps'); plt.ylabel('elapsed_time (mins)')
plt.grid(); plt.show()

## Test data loader

In [ ]:
%%time
# Test dataset
test_cfg = cfg["test_data_loader"]
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open(cached=False)  # to prevent run out of memory
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset, shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"], num_workers=test_cfg["num_workers"],
                             pin_memory=False
                            )
print(test_dataset)

## Prediction loop

In [ ]:
model.eval()
torch.set_grad_enabled(False)

# store information for evaluation
future_coords_offsets_pd = []
timestamps = []
confidences_list = []
agent_ids = []
memorys_pred = []
t0 = time.time()
times_pred = []
iterations_pred = []

for i, data in enumerate(tqdm_notebook(test_dataloader, mininterval=5.)):
    
    try:
        _, preds, confidences = forward(data, model, device, compute_loss=True)

        rotation = data["world_from_agent"][:, :2, :2].float().to(device)
        preds = torch.sum(preds[:, :, :, None, :] * rotation[:, None, None, :, :], dim=-1).cpu().numpy()
        # same as: preds = torch.einsum('bmti,bji->bmtj', preds, rotation).cpu().numpy()

        future_coords_offsets_pd.append(preds.copy())
        confidences_list.append(confidences.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy()) 

        if i%100 == 0:
            t = ((time.time() - t0) / 60)
            print('%4d'%i, '%6.2fmins'%t, end=' | ')
            mem = memory()
            iterations_pred.append(i)
            memorys_pred.append(mem)
            times_pred.append(t)
    except:
        continue
print('Total timespent: %6.2fmins'%((time.time() - t0) / 60))
memory()

## Submission file creation

In [ ]:
%%time
# create submission to submit to Kaggle
pred_path = 'submission.csv'
write_pred_csv(
    pred_path,
    timestamps=np.concatenate(timestamps),
    track_ids=np.concatenate(agent_ids),
    coords=np.concatenate(future_coords_offsets_pd),
    confs=np.concatenate(confidences_list),
)

In [ ]:
df_sub = pd.read_csv(pred_path)
df_sub = df_sub.set_index(['timestamp', 'track_id'])
display(df_sub)

## Prediction plotting functions

In [ ]:
# plot functions
import matplotlib.patches as mpatches

def row_to_confs(row):
    return [row[f'conf_{i}'] for i in range(3)]
def row_to_coords(row):
    return row[3:].values.reshape(3, 50, 2)

# here I use matplotlib default colors
cmap = plt.get_cmap("tab10")
matplotlib_colors_in_rgb_int = [
    [int(255 * x) for x in cmap(i)[:3]] for i in range(10)
]

def generate_image_predicted_trajectory(dataset, df_sub, index):
    data = dataset[index]
    im = data['image'].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    row = df_sub.loc[(data['timestamp'], data['track_id'])]
    # note submission coordinate system = world - centroid
    predicted_target_positions_in_sub = row_to_coords(row)
    predicted_target_positions_in_world = predicted_target_positions_in_sub + data['centroid']
    for i, coords in enumerate(predicted_target_positions_in_world):
        target_positions_pixels = transform_points(coords, data['raster_from_world'])
        draw_trajectory(im, target_positions_pixels, rgb_color=matplotlib_colors_in_rgb_int[i])
    return im, row_to_confs(row)

def plot_predicted_trajectory(dataset, df_sub, indices, width=12, height=4, n_cols=3, title=''):
    if not isinstance(indices, (list, np.ndarray)):
        indices = [indices]
    n_rows = len(indices) // n_cols + len(indices) % n_cols
    plt.figure(figsize=(width, height*n_rows))
    for k, index in enumerate(indices):
        plt.subplot(n_rows, n_cols, 1+k).set_title(str(index))
        im, confs = generate_image_predicted_trajectory(dataset, df_sub, index)
        patches = [mpatches.Patch(color=cmap(m), label='%.3f'%conf) for m, conf in enumerate(confs)]
        plt.imshow(im, origin='lower')
        plt.legend(handles=patches)
    if title:
        plt.suptitle(title)
    plt.show()

In [ ]:
plot_predicted_trajectory(test_dataset, df_sub, [18431], width=6, height=6, n_cols=1)

In [ ]:
i_plots = np.random.randint(len(test_dataset), size=9)
plot_predicted_trajectory(test_dataset, df_sub, i_plots)